In [2]:
!pip install psycopg2-binary

import pandas as pd


In [3]:
import pandas as pd
import psycopg2


In [13]:
import psycopg2
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="a9Am7Yy5r9_T7h4OF2GN",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor() 
print("✅ Connection successful!")


✅ Connection successful!


In [14]:
query = """
SELECT borough, COUNT(DISTINCT dbn) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY school_count DESC;
"""
df_distribution = pd.read_sql(query, conn)
df_distribution
# Step 3: Run a Test Query
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head() 
print("✅ Counting unique dbn values per borough")



/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_39455/832376122.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_distribution = pd.read_sql(query, conn)
/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_39455/832376122.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ Counting unique dbn values per borough


In [15]:
# Step 4: Average ELL% per borough
query = """
SELECT d.borough,
       ROUND(AVG(s.ell_percent::numeric), 2) AS avg_ell_percent
FROM nyc_schools.high_school_directory AS d
LEFT JOIN nyc_schools.school_demographics AS s
ON d.dbn = s.dbn
GROUP BY d.borough
ORDER BY avg_ell_percent DESC;
"""
df_ell = pd.read_sql(query, conn)
df_ell


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_39455/3057439515.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Brooklyn,NaN
1,Queens,NaN
2,Staten Island,NaN
3,Bronx,NaN
4,Manhattan,7.57


In [9]:
query = """
WITH ranked AS (
    SELECT dir.borough,
           dir.school_name,
           ROUND(d.sped_percent::numeric, 2) AS sped_percent,
           ROW_NUMBER() OVER (PARTITION BY dir.borough ORDER BY d.sped_percent DESC) AS rank
    FROM nyc_schools.school_demographics AS d
    JOIN nyc_schools.high_school_directory AS dir
      ON d.dbn = dir.dbn
    WHERE d.sped_percent IS NOT NULL
)
SELECT borough, school_name, sped_percent
FROM ranked
WHERE rank <= 3;
"""

df_top3 = pd.read_sql(query, conn)
df_top3




/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_39455/1469301735.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top3 = pd.read_sql(query, conn)


,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


In [17]:
# ==== FINAL RESULTS SUMMARY ====
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2)

print("✅ Final Results Summary")

# Schools per Borough
print("\n— Schools per Borough —")
try:
    display(df_schools)
except NameError:
    print("df_schools not found. (Run the schools-per-borough cell.)")

# Average ELL% per Borough
print("\n— Average ELL% per Borough —")
try:
    display(df_ell_nulls)
except NameError:
    print("df_ell_nulls not found. (Run the ELL cell with LEFT JOIN.)")


✅ Final Results Summary

— Schools per Borough —
df_schools not found. (Run the schools-per-borough cell.)

— Average ELL% per Borough —
df_ell_nulls not found. (Run the ELL cell with LEFT JOIN.)


### Notes on Data Availability
1. The ELL (English Language Learner) data is only available for Manhattan in this dataset.  
2. Other boroughs show NULL values because demographic data is missing.  
3. This indicates a data gap rather than an error in the queries.

**Additional Note:**  
The dataset used in this analysis has limited demographic coverage.  
ELL and sped_percent values are only available for Manhattan, while other boroughs show NULL due to missing records.  
This does not indicate a query error — it reflects a limitation in the source data.


